## Visualization utilities

In [1]:
#@markdown We implemented some functions to visualize the face landmark detection results. <br/> Run the following cell to activate the functions.
import mediapipe as mp
from mediapipe import solutions
from mediapipe.framework.formats import landmark_pb2
import numpy as np
import matplotlib.pyplot as plt


def draw_landmarks_on_image(rgb_image, detection_result):
  face_landmarks_list = detection_result.face_landmarks
  annotated_image = np.copy(rgb_image)

  # Loop through the detected faces to visualize.
  for idx in range(len(face_landmarks_list)):
    face_landmarks = face_landmarks_list[idx]

    # Draw the face landmarks.
    face_landmarks_proto = landmark_pb2.NormalizedLandmarkList()
    face_landmarks_proto.landmark.extend([
      landmark_pb2.NormalizedLandmark(x=landmark.x, y=landmark.y, z=landmark.z) for landmark in face_landmarks
    ])

    solutions.drawing_utils.draw_landmarks(
        image=annotated_image,
        landmark_list=face_landmarks_proto,
        connections=mp.solutions.face_mesh.FACEMESH_TESSELATION,
        landmark_drawing_spec=None,
        connection_drawing_spec=mp.solutions.drawing_styles
        .get_default_face_mesh_tesselation_style())
    solutions.drawing_utils.draw_landmarks(
        image=annotated_image,
        landmark_list=face_landmarks_proto,
        connections=mp.solutions.face_mesh.FACEMESH_CONTOURS,
        landmark_drawing_spec=None,
        connection_drawing_spec=mp.solutions.drawing_styles
        .get_default_face_mesh_contours_style())
    solutions.drawing_utils.draw_landmarks(
        image=annotated_image,
        landmark_list=face_landmarks_proto,
        connections=mp.solutions.face_mesh.FACEMESH_IRISES,
          landmark_drawing_spec=None,
          connection_drawing_spec=mp.solutions.drawing_styles
          .get_default_face_mesh_iris_connections_style())

  return annotated_image

def plot_face_blendshapes_bar_graph(face_blendshapes):
  # Extract the face blendshapes category names and scores.
  face_blendshapes_names = [face_blendshapes_category.category_name for face_blendshapes_category in face_blendshapes]
  face_blendshapes_scores = [face_blendshapes_category.score for face_blendshapes_category in face_blendshapes]
  # The blendshapes are ordered in decreasing score value.
  face_blendshapes_ranks = range(len(face_blendshapes_names))

  fig, ax = plt.subplots(figsize=(12, 12))
  bar = ax.barh(face_blendshapes_ranks, face_blendshapes_scores, label=[str(x) for x in face_blendshapes_ranks])
  ax.set_yticks(face_blendshapes_ranks, face_blendshapes_names)
  ax.invert_yaxis()

  # Label each bar with values
  for score, patch in zip(face_blendshapes_scores, bar.patches):
    plt.text(patch.get_x() + patch.get_width(), patch.get_y(), f"{score:.4f}", va="top")

  ax.set_xlabel('Score')
  ax.set_title("Face Blendshapes")
  plt.tight_layout()
  plt.show()

## Running inference and visualizing the results

Here are the steps to run face landmark detection using MediaPipe.

Check out the [MediaPipe documentation](https://developers.google.com/mediapipe/solutions/vision/face_landmarker/python) to learn more about configuration options that this task supports.


In [ ]:
import mediapipe as mp

from mediapipe.tasks import python
from mediapipe.tasks.python import vision

import cv2

cap = cv2.VideoCapture(0)
# STEP 2: Create an FaceLandmarker object.
base_options = python.BaseOptions(model_asset_path='./model/face_landmarker.task')
options = vision.FaceLandmarkerOptions(base_options=base_options,
                                       output_face_blendshapes=True,
                                       output_facial_transformation_matrixes=True,
                                       num_faces=1)
detector = vision.FaceLandmarker.create_from_options(options)


while cap.isOpened():
    # STEP 3: Load the input image.
    success, image = cap.read()
    image = mp.Image(image_format=mp.ImageFormat.SRGB, data=image)

    # STEP 4: Detect face landmarks from the input image.
    detection_result = detector.detect(image)
    if (len(detection_result.face_blendshapes) >0):
        print(detection_result.face_blendshapes[0][0])

    # STEP 5: Process the detection result. In this case, visualize it.
    # annotated_image = draw_landmarks_on_image(image.numpy_view(), detection_result)
    # cv2.imshow(cv2.cvtColor(annotated_image, cv2.COLOR_RGB2BGR))

In [ ]:
cap.release()
cv2.destroyAllWindows()

In [ ]:
import cv2
import sys
import mediapipe as mp
import mediapipe as mp

from mediapipe.tasks import python
from mediapipe.tasks.python import vision
from mvm_pb2 import DataChannelMessage,DataChannelMessageType

mp_face_mesh = mp.solutions.face_mesh

def encodeTrackingMessage(blendShapes,keypoints):
    message = DataChannelMessage()
    message.type = DataChannelMessageType.TRACKING_MESSAGE
    if blendShapes is not None and len(blendShapes) > 0 :
        for element in blendShapes :
            blendShape = message.tracking_message.blendShapes.blendShapes.add()
            blendShape.index = element.index
            blendShape.score = element.score
            blendShape.category_name = element.category_name
    if keypoints is not None and len(keypoints) > 0 :
        for landmark in keypoints :
            keypoint = message.tracking_message.keypoints.keypoints.add()
            keypoint.x = landmark.x
            keypoint.y = landmark.y
            keypoint.z = landmark.z

    return message



base_options = python.BaseOptions(model_asset_path='./model/face_landmarker.task')
options = vision.FaceLandmarkerOptions(base_options=base_options,
                                    output_face_blendshapes=True,
                                    output_facial_transformation_matrixes=True,
                                    num_faces=1)
blendShapesDetector = vision.FaceLandmarker.create_from_options(options)
face_mesh = mp.solutions.face_mesh.FaceMesh(
    max_num_faces=1,
    refine_landmarks=True,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5)

cap = cv2.VideoCapture(0)

while cap.isOpened():
    md_blendShapes = None
    md_keypoints = None
    success, image = cap.read()
    # BlendShapes 
    image_1 = mp.Image(image_format=mp.ImageFormat.SRGB, data=image)
    blendShapes_detection_result = blendShapesDetector.detect(image_1)
    if (len(blendShapes_detection_result.face_blendshapes) > 0) : 
        # Pack the serialized data and send it over the socket
        md_blendShapes = blendShapes_detection_result.face_blendshapes[0]
    
    # Keypoints 
    image.flags.writeable = False
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    results = face_mesh.process(image)
    if (len(results.multi_face_landmarks) > 0) :
        md_keypoints = results.multi_face_landmarks[0].landmark

    # Encoding tracking message
    trackingMessage = encodeTrackingMessage(md_blendShapes , md_keypoints).SerializeToString()
    print(trackingMessage)
    # writer.write(trackingMessage)
    await asyncio.sleep(0.05) # wait for 33ms (30fps)

In [ ]:
plot_face_blendshapes_bar_graph(detection_result.face_blendshapes[0])